In [1]:
import os
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation,Conv2D,MaxPooling2D
from keras.layers import Input, Embedding, Add
from keras import layers
from keras import layers
from keras.layers.embeddings import Embedding
from keras.models import Model
from keras.utils import to_categorical
# NLTK
import nltk
from gensim.models import FastText
from gensim.models import Word2Vec
# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score, f1_score
from sklearn import svm
from keras.layers.merge import concatenate
from keras.layers import Bidirectional
# from keras.layers import K, Activation
import keras.layers as K
from keras.layers import Activation
from keras.engine import Layer
from keras.layers import Dense, Input, Embedding, Dropout, Bidirectional, GRU, Flatten, SpatialDropout1D


/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
Using TensorFlow backend.
/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/keras/callbacks/callbacks.py:19: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  from collections import Iterable
/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is dep

In [2]:
Fold_PATH = '/home/shivam/PycharmProjects/Django/Django_Warehouse/todo-django-rest-framework-master/todo_drf/api/Data_text'
# df = pd.read_excel('Data_text/Fakenews/English_TrainVal.xlsx')
df = pd.read_excel(
     os.path.join(Fold_PATH, "Fakenews", "English_TrainVal.xlsx"),
     engine='openpyxl',
)
# devdf = pd.read_excel('Data_text/Fakenews/English_Val.xlsx')
devdf = pd.read_excel(
     os.path.join(Fold_PATH, "Fakenews", "English_Val.xlsx"),
     engine='openpyxl',
)

df = df.dropna()
df = df.drop_duplicates(subset = ['text'],keep = False)
devdf = devdf.dropna()
print(df.info())
print(devdf.info())


# codemixed_embeddings = FastText.load('augmented_ft.bin')
# print(codemixed_embeddings )
embeddings_index = dict()
f = open('/home/shivam/Data/Projects/Fakenews/augmented/glove.6B.200d.txt')

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))


tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=150)
print(data.shape)
sequences_d = tokenizer.texts_to_sequences(devdf['text'])
data_dev = pad_sequences(sequences_d, maxlen=150)
print(data_dev.shape)
vocabulary_size = len(tokenizer.word_index) + 1 # 20648



# create a weight matrix for words in training docs# create
# embedding_matrix = np.zeros((vocabulary_size, 100))
embedding_matrix = np.zeros((vocabulary_size, 200))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector
#         else:
#             embedding_vector = codemixed_embeddings[word]
#             if embedding_vector is not None:
#                 embedding_matrix[index] = embedding_vector


y= df['label'].astype('category').cat.codes
y_dev = devdf['label'].astype('category').cat.codes


def define_model(length, vocabulary_size, embedding_matrix):
    input1 = Input(shape=(length,))
    embedding1 = Embedding(vocabulary_size, 200, weights=[embedding_matrix], trainable=False)(input1)
    conv1 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding1)
    conv1 = Dropout(0.5)(conv1)
    # capsule1 = Capsule(num_capsule=Num_capsule, dim_capsule=Dim_capsule, routings=3, share_weights=True)(conv1)
    flat1 = Flatten()(conv1)

    dense = Dense(32, activation='relu')(flat1)
    output = Dense(2, activation='softmax')(dense)
    model = Model(inputs=[input1], outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    return model

model = define_model(150, vocabulary_size,embedding_matrix)
maxacc =0
for i in range(1,31):
    model.fit([data], y = to_categorical(y), epochs=1, verbose=1)
    acc = model.evaluate(x= [data_dev], y=to_categorical(y_dev))[1]
    print(i, acc)
#     pred = model.predict([data_test,data_test,data_test])
#     prd = np.argmax(pred,axis = 1)
#     print(confusion_matrix(y_test,prd))
#     print(classification_report(y_test,prd))
#     print("Accuracy : ",accuracy_score(prd,y_test))
    if(acc > maxacc):
        print(acc,i)
        maxacc = acc
        model.save_weights('aug_caps_weights_FND_CNN_trnval_HELIOS.h5')




# model = define_model(150, vocabulary_size,embedding_matrix)
# model.load_weights("/home/shivam/PycharmProjects/Django/Django_Warehouse/todo-django-rest-framework-master/todo_drf/api/models/aug_caps_weights_FND_CNN_trnval_HELIOS.h5")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8558 entries, 0 to 8559
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  8558 non-null   int64 
 1   text        8558 non-null   object
 2   label       8558 non-null   object
dtypes: int64(1), object(2)
memory usage: 267.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140 entries, 0 to 2139
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2140 non-null   int64 
 1   text        2140 non-null   object
 2   label       2140 non-null   object
dtypes: int64(1), object(2)
memory usage: 66.9+ KB
None
Loaded 400000 word vectors.
(8558, 150)
(2140, 150)
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150)]             0         
__________

/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/framework/indexed_slices.py:349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if not isinstance(values, collections.Sequence):
/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/training/tracking/data_structures.py:718: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3, and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):


In [3]:
pred_dev = model.predict([data_dev])
# pred_test = model.predict([data_test])
prd_dev = np.argmax(pred_dev,axis = 1)
# prd_test = np.argmax(pred_test,axis = 1)

print(confusion_matrix(y_dev,prd_dev))
# print(classification_report(y_dev,prd))
# print("Accuracy : ",accuracy_score(prd,y_dev))
print(f1_score(y_dev, prd_dev, average='weighted'))

# print(confusion_matrix(y_test,prd_test))
# print(classification_report(y_test,prd))
# print("Accuracy : ",accuracy_score(prd,y_test))
# print(f1_score(y_test, prd, average='weighted'))

/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[1020    0]
 [   0 1120]]
1.0


In [25]:
model.predict(pad_sequences(tokenizer.texts_to_sequences(["This is the test text to be evaluated"]), maxlen=150))
# model.evaluate(x= [data_dev[0]], y=[to_categorical(y_dev[0])])[1]
# model.predict([data_dev])
# print(np.shape(pad_sequences(tokenizer.texts_to_sequences(["This is the test text to be evaluated"]), maxlen=150)))
# print(np.shape([to_categorical(y_dev[0])]))

/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


array([[0.5938684 , 0.40613168]], dtype=float32)

In [4]:
pred_dev = model.predict([[data_dev[0]]])
prd_dev = np.argmax(pred_dev,axis = 1)
# print(pred_dev)
print(prd_dev)
# pred_dev = model.predict([[data_dev[0]]])
# prd_dev = np.argmax(pred_dev,axis = 1)
# # print(pred_dev)
# print(prd_dev)

/home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: in user code:

    /home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/shivam/anaconda3/envs/entailment/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
        return self(x, training=False)
    /home/sh